1단계) 모듈 설치

In [ ]:
!pip install mediapipe

In [61]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import mediapipe as mp
import pickle

In [3]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from keras.utils.np_utils import to_categorical
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.preprocessing import MinMaxScaler

2단계) 데모 동영상에 레이블 만들기

In [ ]:
cap = cv2.VideoCapture('./drive/MyDrive/Colab Notebooks/FinalProject/data/traindata03.avi') #동영상 불러오기

In [ ]:
frames = [] #프레임을 저장할 변수 초기화
while cap.isOpened() :
    ret, frame = cap.read() #영상을 프레임별로 나눕니다.
    if ret == True :
      frames.append(frame)
    else :
      cap.release()

In [ ]:
start = 2350
term = 20
for i in range(start, start + term):
  frame = cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB) #원하는 범위의 프레임을 플로팅하여 확인후 수작업으로 범위 결정
  plt.imshow(frame)
  plt.axis('off')
  plt.show()
  print(i)

레이블 범위

data01 : 11 741 / 964 1566 / 1723 2359 / ~4306

data02 : 0 637 / 821 1423 / 1741 2423 / ~4281

data03 : 26 734 / 920 1537 / 1706 2169 / ~4214

In [ ]:
full_length = 4215 #범위에 따라 입력
start_1 = 26
end_1 = 734
start_2 = 920
end_2 = 1537
start_3 = 1706
end_3 = 2169

In [ ]:
label_sequence = [] #레이블 저장할 변수 초기화

In [ ]:
for i in range(full_length) : #레이블 입력
  if i >= start_1 and i <= end_1 :
    label_sequence.append(1)
  elif i >= start_2 and i <= end_2 :
    label_sequence.append(2)
  elif i >= start_3 and i <= end_3 :
    label_sequence.append(3)
  else :
    label_sequence.append(0)

In [ ]:
label_sequence = np.array(label_sequence)

In [ ]:
print(label_sequence.shape)

In [ ]:
label_sequence = to_categorical(label_sequence, num_classes=4) #카테고리컬 레이블로 변경

In [ ]:
np.save('./drive/MyDrive/Colab Notebooks/FinalProject/data/traintarget03', label_sequence) #저장

3단계) 사진으로부터 Feature 추출

In [ ]:
cap = cv2.VideoCapture('./drive/MyDrive/Colab Notebooks/FinalProject/data/traindata03.avi') #동영상 불러오기

In [ ]:
sequence = [] #전처리 한 데이터를 저장할 변수 초기화

In [ ]:
while cap.isOpened() :
    ret, frame = cap.read()
    if ret == True :
      with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) #mediapipie로 처리한 결과
      x=  []
      if results.pose_landmarks: #처리하여 랜드마크가 존재할 경우 각 랜드마크를 저장
        for data_point in results.pose_landmarks.landmark: 
          n = []
          n.append(data_point.x)
          n.append(data_point.y)
          n.append(data_point.z)
          x.append(n)
      else: #랜드마크가 없을 경우 모두 0으로 처리
        for _ in range(33):
          x.append([0, 0, 0])
      sequence.append(x)
    else :
      cap.release()

In [ ]:
sequence = np.array(sequence) #np.array타입으로 변경

In [ ]:
print(sequence.shape)

In [ ]:
np.save('./drive/MyDrive/Colab Notebooks/FinalProject/data/traindata03', sequence) #저장

4단계) 모델 구축 및 학습, 저장

In [ ]:
temp01 = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/traindata01.npy')
temp02 = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/traindata02.npy')
temp03 = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/traindata03.npy')

In [ ]:
print(temp01.shape)
print(temp02.shape)
print(temp03.shape)

In [ ]:
train = np.concatenate((temp01, temp02, temp03), axis = 0) #전처리한 데이터를 모두 합칩니다

In [ ]:
print(train.shape)

In [ ]:
temp01 = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/traintarget01.npy')
temp02 = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/traintarget02.npy')
temp03 = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/traintarget03.npy')

In [ ]:
target = np.concatenate((temp01, temp02, temp03), axis = 0) #레이블도 모두 합칩니다

In [ ]:
print(target.shape)

In [ ]:
model = Sequential()
model.add(Conv1D(32, 6, activation = 'relu', input_shape = (33, 3)))
model.add(Conv1D(16, 6))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(16))
model.add(Dense(4, activation = 'sigmoid'))

In [ ]:
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
model.fit(train, target, batch_size = 200, epochs = 10)

In [ ]:
model.save('./drive/MyDrive/Colab Notebooks/FinalProject/data/pretrained.h5')

5단계) 기존 영상으로 테스트

In [56]:
model = load_model('./drive/MyDrive/Colab Notebooks/FinalProject/data/pretrained.h5')

In [57]:
cap = cv2.VideoCapture('./drive/MyDrive/Colab Notebooks/FinalProject/data/traindata02.avi')

In [ ]:
while cap.isOpened() :
    ret, frame = cap.read()
    if ret == True :
      with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) #mediapipie로 처리한 결과
      x = []
      if results.pose_landmarks:
        for data_point in results.pose_landmarks.landmark:
          n = []
          n.append(data_point.x)
          n.append(data_point.y)
          n.append(data_point.z)
          x.append(n)
      else:
        for _ in range(33):
          x.append([0, 0, 0])
      print(model.predict(np.array([x]))) #전처리한 데이터를 바로바로 예측
    else :
      cap.release()

6단계) 인터넷 크롤링 DB 클러스터링

In [4]:
path_dir = './drive/MyDrive/Colab Notebooks/FinalProject/data/padded/' #DB 폴더 경로 선언

In [5]:
file_list = os.listdir(path_dir) #폴더 내 모든 파일을 list화

In [ ]:
print(len(file_list))

6-1) 크롤링 DB 데이터 전처리

In [7]:
train = []

In [8]:
file_count = file_list.copy() #루프에 사용하기 위해 파일 리스트 복사

In [ ]:
for file in file_count :
  try :
    frame = cv2.imread(path_dir + file)
    with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
      results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) #mediapipie로 처리한 결과
    x = []
    if results.pose_landmarks: #처리하여 랜드마크가 존재할 경우 각 랜드마크를 저장
      for data_point in results.pose_landmarks.landmark: 
        n = []
        n.append(data_point.x)
        n.append(data_point.y)
        n.append(data_point.z)
        x.append(n)
      train.append(x)
    else: #랜드마크가 없을 경우 배제
      file_list.remove(file)
  except :
    pass

In [ ]:
print(len(file_list)) 

In [62]:
with open("file_list.pkl","wb") as f: #랜드마크가 존재하는 데이터(6-3 확인용)
    pickle.dump(file_list, f)

In [11]:
train = np.array(train)

In [ ]:
print(train.shape)

In [13]:
np.save('./drive/MyDrive/Colab Notebooks/FinalProject/data/3000images', train) #테스트 용 랜드마크값 저장

In [14]:
scaler = MinMaxScaler() #포즈만 잡을 수 있도록 평준화

In [15]:
for i in range(len(train)):
  train[i] = scaler.fit_transform(train[i])

In [17]:
remove_feature = [21, 20, 17, 16, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1] #손가락, 입, 눈 등 필요 없는 Feature 제거

In [18]:
for i in remove_feature :
  train = np.delete(train, i, axis = 1)

In [ ]:
print(train.shape)

In [20]:
train = train.reshape(train.shape[0], train.shape[1] * train.shape[2]) #Clustering에 알맞게 데이터 reshape

In [ ]:
print(train.shape)

In [22]:
np.save('./drive/MyDrive/Colab Notebooks/FinalProject/data/3000cl_feat', train) #저장

6-2) 전처리한 데이터 Clustering

In [23]:
train = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/3000cl_feat.npy') #저장된 feature 불러오기

In [26]:
clusterMethod = MeanShift(bandwidth = 1.0, cluster_all = False) #클러스터링
clustering = clusterMethod.fit_predict(train)

In [ ]:
print(clustering) #클러스터링 결과 확인
print(clustering.max()) 

In [28]:
np.save('./drive/MyDrive/Colab Notebooks/FinalProject/data/clustered', clustering) #클러스터 저장

6-3) Clustered 데이터 확인

In [29]:
clustering = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/clustered.npy') #클러스터 값 불러오기

In [24]:
frames = [] #확인하기 위해 이미지 다시 불러오기위한 변수값 초기화

In [63]:
with open("file_list.pkl","rb") as f:
    file_list = pickle.load(f)

In [25]:
for file in file_list: #이미지 목록 생성
  frame = cv2.imread(path_dir + file)
  frames.append(frame)

In [ ]:
label = np.zeros(train.shape[0])
print(label.shape)

In [ ]:
for i in range(len(clustering)) : #지정한 클러스터의 사진들 보여주기
  if clustering[i] == 2 :
    imgRGB = cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB)
    plt.imshow(imgRGB)
    plt.axis('off')
    plt.show()

6-4) Clustered 데이터 레이블링

In [ ]:
for i in range(clustering.max()) :  #샘플 개수가 10개를 초과한  클러스터만 유의미한 것으로 판정
  if len(clustering[clustering == i]) > 10 :
    label[clustering == i] = 1
    print(i, len(clustering[clustering == i]))

In [ ]:
print(len(label[label == 1])) #샘플 개수 확인
print(len(label[label == 0]))

In [35]:
np.save('./drive/MyDrive/Colab Notebooks/FinalProject/data/cl_label', label) #레이블 저장

7단계) Clustered Data로 학습시켜보기

In [37]:
model = Sequential() #기존 모델과 유사하지만 결과만 1(좋은 포즈임), 0(그렇지 않음)로 출력되게 변경
model.add(Conv1D(32, 6, activation = 'relu', input_shape = (33, 3)))
model.add(Conv1D(16, 6))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(16))
model.add(Dense(2, activation = 'sigmoid'))

In [ ]:
print(model.summary())

In [39]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [40]:
train = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/3000images.npy')

In [41]:
target = np.load('./drive/MyDrive/Colab Notebooks/FinalProject/data/cl_label.npy')
target = to_categorical(target, num_classes=2) #categorical label 화

In [ ]:
print(train.shape) #입출력 데이터 형태 재확인
print(target.shape)

In [ ]:
model.fit(train, target, batch_size = 200, epochs = 80) #학습

In [46]:
model.save('./drive/MyDrive/Colab Notebooks/FinalProject/data/pretrained_cl.h5') #모델 저장